- **Data from**: https://www.kaggle.com/competitions/plant-seedlings-classification/data
- **Kaggle notebook**: https://www.kaggle.com/code/ducnh279/efficientnet-3-heads-cosine-decay

In [1]:
import os
import time

import pandas as pd
from PIL import Image
import multiprocessing as mp

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from transformers import get_cosine_schedule_with_warmup

import timm

# 1. General Settings

In [2]:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

In [3]:
TRAIN_DIR =  '../input/plant-seedlings-classification/train/'
TEST_DIR = '/kaggle/input/plant-seedlings-classification/test'
SUB_DIR = '/kaggle/input/plant-seedlings-classification/sample_submission.csv'

In [4]:
TRAIN_DIR =  '../input/plant-seedlings-classification/train/'
TEST_DIR = '/kaggle/input/plant-seedlings-classification/test'
SUB_DIR = '/kaggle/input/plant-seedlings-classification/sample_submission.csv'
SEED = 279
TRAIN_BS = 16
TEST_BS = 100
NUM_CLASSES = 128
EMBEDDING_SIZE = 1280
NUM_EPOCHS = 7
LEARNING_RATE = 0.0003
WEIGHT_DECAY = 0
WARMUP_EPOCHS = 0
LOGGING_INTERVAL = 100
N_CORES = mp.cpu_count()

In [5]:
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
set_seed()

In [6]:
if torch.cuda.is_available():
    DEVICE = torch.device(type='cuda')
else:
    DEVICE = torch.device('cpu')
print(f'Using device: {DEVICE}')

Using device: cuda


# 2. Create an image dataframe with labels

In [7]:
class_label_names = os.listdir(TRAIN_DIR)
print(class_label_names)

train_data = []
for label_index, label_name in enumerate(class_label_names):
    fnames = os.listdir(os.path.join(TRAIN_DIR, label_name)) 
    for fname in fnames:
        train_data.append([os.path.join(label_name, fname), label_index])

train_df = pd.DataFrame(train_data, columns=['file_path', 'label'])
train_df.tail()

['Scentless Mayweed', 'Common wheat', 'Charlock', 'Black-grass', 'Sugar beet', 'Loose Silky-bent', 'Maize', 'Cleavers', 'Common Chickweed', 'Fat Hen', 'Small-flowered Cranesbill', 'Shepherds Purse']


,file_path,label
4745,Shepherds Purse/f0127f70d.png,11
4746,Shepherds Purse/179cedc9e.png,11
4747,Shepherds Purse/a0ec33869.png,11
4748,Shepherds Purse/33010c8cb.png,11
4749,Shepherds Purse/aad81b27b.png,11


In [8]:
test_files = os.listdir(TEST_DIR)
test_df = pd.DataFrame({'file_path': test_files})
test_df['label'] = 9999

# 3. Datasets and DataLoaders

In [9]:
class PlantSeedingDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.file_path = df['file_path']
        self.y = df['label']
    
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir, self.file_path[index]))
        if self.transform is not None:
            img = self.transform(img)
        label = self.y[index]
        return img, label
    
    def __len__(self):
        return self.y.shape[0]

In [10]:
def convert_4_channel_to_3_channel(image):
    """
    Convert 4-channel RGBA image to 3-channel RGB image
    """
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    return image

train_transforms = transforms.Compose([
    transforms.Lambda(convert_4_channel_to_3_channel), 
    transforms.Resize(size=(250, 250), interpolation=InterpolationMode.BILINEAR),
    transforms.RandomRotation(degrees=(-180, 180)),
    transforms.RandomAffine(degrees=0, translate=(0.3, 0.3), scale=(0.7, 1.3)),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transforms = transforms.Compose([
    transforms.Lambda(convert_4_channel_to_3_channel), 
    transforms.Resize(size=(224, 224), interpolation=InterpolationMode.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [11]:
train_dataset = PlantSeedingDataset(df=train_df,
                                    img_dir=TRAIN_DIR,
                                    transform=train_transforms)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=TRAIN_BS,
                          drop_last=True,
                          shuffle=True,
                          num_workers=N_CORES)


test_dataset = PlantSeedingDataset(df=test_df,
                                   img_dir=TEST_DIR,
                                   transform=test_transforms)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=TEST_BS,
                         drop_last=False,
                         shuffle=False,
                         num_workers=N_CORES)

# 4. Model initialization and training setup

In [12]:
class Net(nn.Module):
    def __init__(self, num_classes, embedding_size):
        super(Net, self).__init__()
        self.num_classes = num_classes
        self.embedding_size = embedding_size
        self.backbone = timm.create_model(
            'efficientnet_b1', 
            pretrained=True, 
            num_classes=self.num_classes
        )
        
        self.backbone.classifier = nn.Sequential(
            nn.Linear(self.embedding_size, 256),
            nn.BatchNorm1d(256),
            nn.PReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.PReLU(),
            nn.Linear(128, self.num_classes)
        )
        
    def forward(self, x):
        logits = self.backbone(x)
        return logits

In [13]:
set_seed(SEED)
model = Net(num_classes=NUM_CLASSES, embedding_size=EMBEDDING_SIZE)
model.to(DEVICE);

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b1-533bc792.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-533bc792.pth


In [14]:
optimizer = optim.AdamW(
    model.parameters(), 
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)

scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=WARMUP_EPOCHS,
    num_training_steps=len(train_loader)*NUM_EPOCHS
)

scaler = GradScaler()

# 5. Fine-tuning

In [15]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    
    for batch_idx, (features, target) in enumerate(train_loader):
        features = features.to(DEVICE)
        target = target.to(DEVICE)

        with autocast():
            logits = model(features)
            loss = F.cross_entropy(logits, target, reduction='mean')

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()
        
        
        if not batch_idx % LOGGING_INTERVAL:
            print(
                f'Epoch: {epoch + 1}/{NUM_EPOCHS}'
                f' | Batch: {batch_idx}/{len(train_loader)}'
                f' | Loss: {loss:.4f}'
            )

            
elapsed = (time.time() - start_time) / 60
print(f'Total training time: {elapsed:.3f} min')

Epoch: 1/7 | Batch: 0/296 | Loss: 4.9934
Epoch: 1/7 | Batch: 100/296 | Loss: 1.3283
Epoch: 1/7 | Batch: 200/296 | Loss: 0.8817
Epoch: 2/7 | Batch: 0/296 | Loss: 0.1807
Epoch: 2/7 | Batch: 100/296 | Loss: 0.3612
Epoch: 2/7 | Batch: 200/296 | Loss: 0.2435
Epoch: 3/7 | Batch: 0/296 | Loss: 0.4535
Epoch: 3/7 | Batch: 100/296 | Loss: 0.0638
Epoch: 3/7 | Batch: 200/296 | Loss: 0.3376
Epoch: 4/7 | Batch: 0/296 | Loss: 0.1812
Epoch: 4/7 | Batch: 100/296 | Loss: 0.3435
Epoch: 4/7 | Batch: 200/296 | Loss: 0.5069
Epoch: 5/7 | Batch: 0/296 | Loss: 0.1083
Epoch: 5/7 | Batch: 100/296 | Loss: 0.0577
Epoch: 5/7 | Batch: 200/296 | Loss: 0.1180
Epoch: 6/7 | Batch: 0/296 | Loss: 0.0244
Epoch: 6/7 | Batch: 100/296 | Loss: 0.0728
Epoch: 6/7 | Batch: 200/296 | Loss: 0.0177
Epoch: 7/7 | Batch: 0/296 | Loss: 0.1290
Epoch: 7/7 | Batch: 100/296 | Loss: 0.0237
Epoch: 7/7 | Batch: 200/296 | Loss: 0.1128
Total training time: 7.510 min


# 6. Inference

In [16]:
preds = []
model.eval()

with torch.inference_mode():
    for features, target in test_loader:
        features = features.to(DEVICE)
        target = target.to(DEVICE)
        
        with autocast():
            logits = model(features)
            
        y_pred = torch.softmax(logits, dim=1).argmax(dim=1).detach().cpu()
        
        for pred in y_pred:
            preds.append(pred.item())

# 7. Submit

In [17]:
map_labels = {}
for idx, i in enumerate(class_label_names):
    map_labels[idx] = i

In [18]:
sub_df = pd.read_csv(SUB_DIR)
sub_df['file'] = test_df['file_path']
sub_df['species'] = pd.Series(preds).map(map_labels)
sub_df.to_csv('submission.csv', index=False)
print('Submission file saved!')
sub_df

Submission file saved!


,file,species
0,fd87b36ae.png,Loose Silky-bent
1,0e8492cb1.png,Sugar beet
2,8d6acbe9b.png,Common Chickweed
3,54b3afd58.png,Cleavers
4,6049234e6.png,Fat Hen
...,...,...
789,4c7838de4.png,Common Chickweed
790,fda39e16f.png,Loose Silky-bent
791,da4ed3a28.png,Charlock
792,a83820a2c.png,Sugar beet
